In [40]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras import models, layers
from keras.utils import to_categorical


In [41]:
absolutepath = '/Users/helenamehler/PycharmProjects/fallDetection/dataSets/FallAllD_Labelled'

In [42]:
x_train = []
y_train = []
x_test = []
y_test = []

for root, dirs, files in os.walk(absolutepath):
    for file in files:
        if "acc" in file:
            filepath = os.path.join(root, file)
            df = pd.read_csv(filepath)
            features = df[['accelerometer_x', 'accelerometer_y', 'accelerometer_z']]
            label = df['fall'][0]

            if np.random.rand(1)[0] < 0.2:
                x_test.append(features.values)
                y_test.append(label)
            else:
                x_train.append(features.values)
                y_train.append(label)


In [43]:
scaler = MinMaxScaler()

x_train = np.array(x_train)
test = x_train.reshape(-1, x_train.shape[-1])
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)

x_test = np.array(x_test)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [44]:
y_trainCat = to_categorical(y_train, num_classes=2)

In [45]:
input = (x_train.shape[1], x_train.shape[2])
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=4, activation='relu', input_shape=input, padding='same'))

model.add(layers.MaxPooling1D(pool_size=4, strides=4))

model.add(layers.Conv1D(12, kernel_size=4, activation='relu',padding='same'))

model.add(layers.MaxPooling1D(pool_size=4, strides=4))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
batch = 80
epochs = 20
iterationsPerEp = x_train.shape[0] // batch
model.fit(x_train, y_trainCat, batch_size=batch, epochs=epochs, steps_per_epoch=iterationsPerEp)

Epoch 1/20
15/15 [==============================] - 1s 20ms/step - loss: 0.6301 - accuracy: 0.6933
Epoch 2/20
15/15 [==============================] - 0s 21ms/step - loss: 0.5704 - accuracy: 0.7285
Epoch 3/20
15/15 [==============================] - 0s 20ms/step - loss: 0.5311 - accuracy: 0.7201
Epoch 4/20
15/15 [==============================] - 0s 20ms/step - loss: 0.4715 - accuracy: 0.7619
Epoch 5/20
15/15 [==============================] - 0s 21ms/step - loss: 0.4663 - accuracy: 0.7845
Epoch 6/20
15/15 [==============================] - 0s 19ms/step - loss: 0.4229 - accuracy: 0.7970
Epoch 7/20
15/15 [==============================] - 0s 20ms/step - loss: 0.3960 - accuracy: 0.8104
Epoch 8/20
15/15 [==============================] - 0s 20ms/step - loss: 0.3738 - accuracy: 0.8421
Epoch 9/20
15/15 [==============================] - 0s 19ms/step - loss: 0.3645 - accuracy: 0.8421
Epoch 10/20
15/15 [==============================] - 0s 20ms/step - loss: 0.3457 - accuracy: 0.8438
Epoch 11/

In [47]:
predictions = model.predict(x_test)
yPredClasses = np.argmax(predictions, axis=1)
confMat = confusion_matrix(y_test, yPredClasses)
confMat

11/11 [==============================] - 0s 4ms/step


array([[243,   8],
       [ 28,  72]])

In [49]:
print('Accuracy', accuracy_score(y_test, yPredClasses))
print('F1-Score', f1_score(y_test, yPredClasses))

Accuracy 0.8974358974358975
F1-Score 0.7999999999999999
